A fair value gap (FVG) is a concept where there is an imbalance in price action, typically identified on a price chart. It represents a region where a significant price movement has left a "gap" due to the lack of sufficient trading activity or liquidity.
Look for areas on a price chart where price movement has created a gap that hasn't been filled by subsequent price action. Here's how you can implement this concept in Python using historical price data (e.g., Open, High, Low, Close data):

Identify a significant price movement (e.g., a large bullish or bearish candle).
Check if there is a gap between the high of one candle and the low of a subsequent candle (or vice versa).
Mark this region as a fair value gap.
As time goes on, indicator must check if the gap is still open. Only open gaps should be considered for trades. Think about the lists of fvg we got, we will use the FVG_Start to set pending orders and set stop loss is FVG_End.

TODO: Exposure time is at 99% lol must check logic prolly tp sl too wide and optimize() just overfit

TODO: Fix dataframe columns to match format expected by backtesting.py's plot() method to debug

In [35]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from backtesting import Backtest, Strategy

def load_data(symbol, timeframe, start_date, end_date):
    mt5.initialize()
    timezone = mt5.TIMEFRAME_M1 if timeframe == '1min' else \
              mt5.TIMEFRAME_M5 if timeframe == '5min' else \
              mt5.TIMEFRAME_M15 if timeframe == '15min' else \
              mt5.TIMEFRAME_M30 if timeframe == '30min' else \
              mt5.TIMEFRAME_H1 if timeframe == '1hour' else \
              mt5.TIMEFRAME_H4 if timeframe == '4hour' else \
              mt5.TIMEFRAME_D1 if timeframe == 'daily' else None

    if timezone is None:
        raise ValueError("Unsupported timeframe")

    rates = mt5.copy_rates_range(symbol, timezone, start_date, end_date)
    mt5.shutdown()

    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    data.set_index('time', inplace=True)

    data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)

    return data

def is_gap_filled(gap, current_low, current_high):
    if gap['FVG_Type'] == 'Bullish':
        return current_low <= gap['FVG_Start']
    else:  # Bearish
        return current_high >= gap['FVG_Start']

def identify_and_track_fair_value_gaps(data, min_gap_size=0.0001):
    fvg_data = []
    open_gaps = []

    for i in range(2, len(data)):
        prev_high = data.iloc[i - 2]['High']
        prev_low = data.iloc[i - 2]['Low']

        current_high = data.iloc[i]['High']
        current_low = data.iloc[i]['Low']
        current_datetime = data.index[i]

        # Check for a bullish gap
        if current_low > prev_high:
            gap_size = current_low - prev_high
            if gap_size > min_gap_size:
                new_gap = {
                    'index': i,
                    'FVG_Type': 'Bullish',
                    'FVG_Start': prev_high,
                    'FVG_End': current_low,
                    'Start_Time': current_datetime,
                    'Filled': False
                }
                fvg_data.append(new_gap)
                open_gaps.append(new_gap)

        # Check for a bearish gap
        if prev_low > current_high:
            gap_size = prev_low - current_high
            if gap_size > min_gap_size:
                new_gap = {
                    'index': i,
                    'FVG_Type': 'Bearish',
                    'FVG_Start': prev_low,
                    'FVG_End': current_high,
                    'Start_Time': current_datetime,
                    'Filled': False
                }
                fvg_data.append(new_gap)
                open_gaps.append(new_gap)

        # Check if any open gaps have been filled
        open_gaps = [gap for gap in open_gaps if not is_gap_filled(gap, current_low, current_high)]

    return pd.DataFrame(open_gaps)

class FVGStrategy(Strategy):
    sl_pips = 0.00030  # 30 pips stop loss
    tp_pips = 0.00030  # 30 pips take profit

    def init(self):
        start_date = datetime(2020, 1, 1)
        end_date = datetime(2022, 12, 31)
        data_1h = load_data(symbol, '1hour', start_date, end_date)
        data_4h = load_data(symbol, '4hour', start_date, end_date)
        data_daily = load_data(symbol, 'daily', start_date, end_date)

        fvgs_1h = identify_and_track_fair_value_gaps(data_1h)
        fvgs_4h = identify_and_track_fair_value_gaps(data_4h)
        fvgs_daily = identify_and_track_fair_value_gaps(data_daily)
        self.fvgs = pd.concat([fvgs_1h, fvgs_4h, fvgs_daily])

    def next(self):
            current_price = self.data.Close[-1]
            tp_buffer = 0.000001
            position_size = 0.10  # Fixed position size

            # Check FVGs
            for _, gap in self.fvgs.iterrows():
                #print(f"Checking FVG: {gap}") 
                if gap['FVG_Type'] == 'Bullish' and current_price >= gap['FVG_End'] and not gap['Filled']:
                    #print(f"Placing sell order at {current_price} with SL: {round(current_price + sl_pips + tp_buffer, 5)}, TP: {round(current_price - tp_pips + tp_buffer, 5)}, Size: {position_size}") 
                    self.sell(size=position_size, sl=round(current_price + self.sl_pips + tp_buffer, 5), tp=round(current_price - self.tp_pips + tp_buffer, 6))
                    gap['Filled'] = True  
                elif gap['FVG_Type'] == 'Bearish' and current_price <= gap['FVG_End'] and not gap['Filled']:
                    #print(f"Placing buy order at {current_price} with SL: {round(current_price - sl_pips - tp_buffer, 5)}, TP: {round(current_price + tp_pips - tp_buffer, 5)}, Size: {position_size}")
                    self.buy(size=position_size, sl=round(current_price - self.sl_pips - tp_buffer, 5), tp=round(current_price + self.tp_pips - tp_buffer, 6))
                    gap['Filled'] = True

# Backtesting setup
symbol = "EURUSD"
backtest_start_date = datetime(2023, 1, 1)
backtest_end_date = datetime(2023, 12, 31)
data_30m = load_data(symbol, '30min', backtest_start_date, backtest_end_date)

In [36]:
bt = Backtest(data_30m, FVGStrategy, cash=10000, commission=.0002, exclusive_orders=False)
stats = bt.run()
print(stats)

Start                     2023-01-02 00:00:00
End                       2023-12-29 23:30:00
Duration                    361 days 23:30:00
Exposure Time [%]                    99.98391
Equity Final [$]                   459.747392
Equity Peak [$]                  10001.827348
Return [%]                         -95.402526
Buy & Hold Return [%]                3.259639
Return (Ann.) [%]                  -94.888093
Volatility (Ann.) [%]                0.187476
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -95.403366
Avg. Drawdown [%]                  -95.403366
Max. Drawdown Duration      361 days 22:00:00
Avg. Drawdown Duration      361 days 22:00:00
# Trades                               906900
Win Rate [%]                        32.310949
Best Trade [%]                        0.52699
Worst Trade [%]                     -0.330763
Avg. Trade [%]                    

In [37]:

stats = bt.optimize(
    sl_pips=[float(x) / 10000 for x in range(10, 51, 5)],  # Convert to list of floats
    tp_pips=[float(x) / 10000 for x in range(10, 51, 5)],  # Convert to list of floats
    maximize='SQN',          
)

print(stats)

Start                     2023-01-02 00:00:00
End                       2023-12-29 23:30:00
Duration                    361 days 23:30:00
Exposure Time [%]                    99.98391
Equity Final [$]                  10703.67844
Equity Peak [$]                  10926.269004
Return [%]                           7.036784
Buy & Hold Return [%]                3.259639
Return (Ann.) [%]                    6.997809
Volatility (Ann.) [%]               10.286849
Sharpe Ratio                         0.680268
Sortino Ratio                        1.059536
Calmar Ratio                         0.952166
Max. Drawdown [%]                   -7.349358
Avg. Drawdown [%]                   -0.680715
Max. Drawdown Duration      164 days 13:30:00
Avg. Drawdown Duration        6 days 00:19:00
# Trades                                68557
Win Rate [%]                        56.691512
Best Trade [%]                       0.506205
Worst Trade [%]                     -0.504127
Avg. Trade [%]                    